In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from glob2 import glob
import folium
from folium.plugins import FeatureGroupSubGroup
from folium.plugins import FastMarkerCluster
from folium.plugins import MiniMap
from folium.plugins import DualMap

### Load observations

In [2]:
#load data
data_folder = '../data/observations_final'
df = pd.concat([pd.read_csv(f).assign(challenge=f.replace('.csv','')) for f in glob(data_folder+'/CNC_London_*.csv')])

In [3]:
#create year 
df['year']=df['time_observed_at'].str[0:4].astype('int64')

In [4]:
df.shape

(13750, 40)

In [5]:
df.head()

,id,observed_on_string,time_observed_at,created_time_zone,created_at,updated_at,description,user_id,user_login,quality_grade,...,taxon_rank,taxon_parent_id,taxon_native,taxon_endemic,taxon_threatened,taxon_search_rank,taxon_observations,identifications,challenge,year
0,11479007,2018-04-27 7:52:34 am BST,2018-04-27T07:52:34+00:00,Etc/UTC,2018-04-27T07:06:17+00:00,2018-04-27T09:26:56+00:00,NaN,159021,muki,research,...,species,204212,True,False,False,27624,27624,"[{'user_id': 353381, 'category': 'improving', ...",../data/observations_final\CNC_London_2018,2018
1,11480212,2018-04-27 8:39:41 am BST,2018-04-27T08:39:41+01:00,Europe/London,2018-04-27T09:02:52+01:00,2019-01-29T21:16:34+00:00,NaN,664459,lucyrobinsonnhm,research,...,variety,3017,False,False,False,43329,43329,"[{'user_id': 29729, 'category': 'supporting', ...",../data/observations_final\CNC_London_2018,2018
2,11480902,Fri Apr 27 2018 08:34:31 GMT+0100 (GMT+1),2018-04-27T08:34:31+02:00,Europe/Amsterdam,2018-04-27T10:42:01+02:00,2018-04-27T13:01:49+02:00,NaN,908315,tess26,casual,...,species,56095,False,False,False,2813,2813,"[{'user_id': 908315, 'category': 'leading', 'd...",../data/observations_final\CNC_London_2018,2018
3,11481097,Fri Apr 27 2018 09:54:35 GMT+0100 (GMT+1),2018-04-27T09:54:35+02:00,Europe/Amsterdam,2018-04-27T10:54:45+02:00,2018-04-27T22:23:41+02:00,NaN,796473,bryonycross,research,...,species,156638,False,False,False,4639,4639,"[{'user_id': 160, 'category': 'improving', 'di...",../data/observations_final\CNC_London_2018,2018
4,11482247,2018-04-27 8:42:21 am BST,2018-04-27T08:42:21+01:00,Europe/London,2018-04-27T11:09:55+01:00,2018-04-27T16:37:31+01:00,NaN,664459,lucyrobinsonnhm,research,...,hybrid,49664,False,False,False,5866,5866,"[{'user_id': 160, 'category': 'supporting', 'd...",../data/observations_final\CNC_London_2018,2018


In [6]:
#check if there are missing latitudes and longitudes
df['latitude'].isna().sum(), df['longitude'].isna().sum()

(0, 0)

### Map

In [7]:
#base map
#coordinates for London
ldn=[51.509865,-0.118092]
m = folium.Map(location=ldn, zoom_start=12, tiles=None, control_scale=True)

#add tile and title 
folium.TileLayer('openstreetmap', name='London OSM').add_to(m)

#add observations
fg = folium.FeatureGroup(name='London (observations)')
m.add_child(fg)

Y2018 = FeatureGroupSubGroup(fg, '2018 observations', show=False)
m.add_child(Y2018)

Y2019 = FeatureGroupSubGroup(fg, '2019 observations', show=False)
m.add_child(Y2019)

Y2020 = FeatureGroupSubGroup(fg, '2020 observations')
m.add_child(Y2020)

df[df['year'].eq(2018)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                             .add_to(Y2018), axis=1)

df[df['year'].eq(2019)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                             .add_to(Y2019), axis=1)

df[df['year'].eq(2020)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                             .add_to(Y2020), axis=1)

#add clusters
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: 'map-marker', markerOpacity: 0, className: 'invisible-marker'});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

clusters=FastMarkerCluster([list(a) for a in zip(np.asarray(df.latitude), np.asarray(df.longitude))],
                           callback=callback,
                           show=False,
                           name='clusters (all years)')
clusters.add_to(m)

#clusters over years
c18=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2018)].latitude),
                                                 np.asarray(df[df['year'].eq(2018)].longitude))],
                           callback=callback,
                           show=False,
                           name='clusters (2018)')
c18.add_to(m)

c19=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2019)].latitude),
                                                 np.asarray(df[df['year'].eq(2019)].longitude))],
                           callback=callback,
                           show=False,
                           name='clusters (2019)')
c19.add_to(m)

c20=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2020)].latitude),
                                                 np.asarray(df[df['year'].eq(2020)].longitude))],
                           callback=callback,
                           name='clusters (2020)')
c20.add_to(m)

#add layer control
folium.LayerControl(collapsed=False).add_to(m)

#add minimap
minimap = MiniMap(width=90, height=90, toggle_display=True, zoom_animation=True)
m.add_child(minimap)

#save map
m.save('../maps/map_london_osm_2018_2020.html')

#call map
#m

### Dual Map - before (2019) and during (2020) pandemic 

In [8]:
#base map
#coordinates for London
ldn=[51.509865,-0.118092]
mdual_osm=DualMap(location=ldn, tiles=None, zoom_start=11, control_scale=True, layout='vertical')

#add tile and title 
folium.TileLayer('openstreetmap', name='London 2020').add_to(mdual_osm.m2)
folium.TileLayer('openstreetmap', name='London 2019').add_to(mdual_osm.m1)


#add observations
fg_m2 = folium.FeatureGroup(name='London (2020)', control=False)
mdual_osm.m2.add_child(fg_m2)

fg_m1 = folium.FeatureGroup(name='London (2019)', control=False)
mdual_osm.m1.add_child(fg_m1)

Y2020 = FeatureGroupSubGroup(fg_m2, 'observations (2020)')
mdual_osm.m2.add_child(Y2020)

Y2019 = FeatureGroupSubGroup(fg_m1, 'observations (2019)')
mdual_osm.m1.add_child(Y2019)


df[df['year'].eq(2019)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                                   .add_to(Y2019), axis = 1)

df[df['year'].eq(2020)].apply(lambda row:folium.CircleMarker(location=[row['latitude'], row['longitude']], 
                                              radius=2, stroke=False, fill_color='orange', fill_opacity=0.6, z_index_offset=1000)
                                                   .add_to(Y2020), axis=1)

#add clusters
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: 'map-marker', markerOpacity: 0, className: 'invisible-marker'});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""

c19_dual=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2019)].latitude),
                                                 np.asarray(df[df['year'].eq(2019)].longitude))],
                           callback=callback,
                           name='clusters (2019)')
c19_dual.add_to(mdual_osm.m1)

c20_dual=FastMarkerCluster([list(a) for a in zip(np.asarray(df[df['year'].eq(2020)].latitude),
                                                 np.asarray(df[df['year'].eq(2020)].longitude))],
                           callback=callback,
                           name='clusters (2020)')
c20_dual.add_to(mdual_osm.m2)


#add layer control
folium.LayerControl(collapsed=False).add_to(mdual_osm)

minimap = MiniMap(width=90, height=90, toggle_display=True, zoom_animation=True)
mdual_osm.add_child(minimap)

mdual_osm.save('../maps/map_london_osm_2019_2020_dualmap.html')

#mdual_osm